# DiagXNet-Lite: Exploratory Data Analysis

This notebook provides a comprehensive exploratory data analysis of the CheXpert dataset for chest X-ray diagnosis.

## Overview
- **Dataset**: CheXpert-small
- **Task**: Multi-label classification of chest X-ray pathologies
- **Labels**: 14 pathological observations
- **Challenge**: Handling uncertain labels (-1)

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("Libraries imported successfully!")

In [ ]:
# Define paths and load configuration
import sys
sys.path.append('../configs')
from config import DATA_ROOT, CHEXPERT_LABELS

# Data paths
train_csv_path = DATA_ROOT / "train" / "train.csv"
valid_csv_path = DATA_ROOT / "valid" / "valid.csv"

print(f"Data root: {DATA_ROOT}")
print(f"Train CSV: {train_csv_path}")
print(f"Valid CSV: {valid_csv_path}")
print(f"Labels: {CHEXPERT_LABELS}")

## 1. Dataset Loading and Basic Information

In [ ]:
# Load the training data
train_df = pd.read_csv(train_csv_path)

print(f"Training dataset shape: {train_df.shape}")
print(f"Number of samples: {len(train_df)}")
print(f"Number of features: {train_df.shape[1]}")

# Display first few rows
print("\nFirst 5 rows:")
train_df.head()

In [ ]:
# Dataset info
print("Dataset Info:")
print(train_df.info())

print("\nColumn names:")
for i, col in enumerate(train_df.columns):
    print(f"{i+1:2d}. {col}")

In [ ]:
# Basic statistics
print("Basic Statistics:")
train_df.describe()

## 2. Patient Demographics Analysis

In [ ]:
# Demographics analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Age distribution
axes[0,0].hist(train_df['Age'].dropna(), bins=50, alpha=0.7, color='skyblue')
axes[0,0].set_title('Age Distribution')
axes[0,0].set_xlabel('Age')
axes[0,0].set_ylabel('Count')

# Sex distribution
sex_counts = train_df['Sex'].value_counts()
axes[0,1].pie(sex_counts.values, labels=sex_counts.index, autopct='%1.1f%%')
axes[0,1].set_title('Sex Distribution')

# Frontal/Lateral distribution
view_counts = train_df['Frontal/Lateral'].value_counts()
axes[1,0].bar(view_counts.index, view_counts.values, color=['lightcoral', 'lightgreen'])
axes[1,0].set_title('View Type Distribution')
axes[1,0].set_ylabel('Count')

# AP/PA distribution
ap_pa_counts = train_df['AP/PA'].value_counts()
axes[1,1].bar(ap_pa_counts.index, ap_pa_counts.values, color=['gold', 'orange'])
axes[1,1].set_title('AP/PA Distribution')
axes[1,1].set_ylabel('Count')

plt.tight_layout()
plt.show()

# Print statistics
print(f"Age statistics:")
print(f"  Mean: {train_df['Age'].mean():.1f} years")
print(f"  Median: {train_df['Age'].median():.1f} years")
print(f"  Range: {train_df['Age'].min():.0f} - {train_df['Age'].max():.0f} years")
print(f"  Missing: {train_df['Age'].isna().sum()} ({train_df['Age'].isna().mean()*100:.1f}%)")

## 3. Label Distribution Analysis

In [ ]:
# Extract pathology labels (excluding metadata columns)
pathology_columns = CHEXPERT_LABELS
pathology_data = train_df[pathology_columns]

print(f"Pathology labels: {pathology_columns}")
print(f"Pathology data shape: {pathology_data.shape}")

# Check unique values in each label column
print("\nUnique values in each pathology column:")
for col in pathology_columns:
    unique_vals = sorted(pathology_data[col].dropna().unique())
    print(f"{col:25s}: {unique_vals}")

In [ ]:
# Label distribution analysis
label_stats = []

for col in pathology_columns:
    col_data = pathology_data[col]
    total = len(col_data)
    
    positive = (col_data == 1.0).sum()
    negative = (col_data == 0.0).sum()
    uncertain = (col_data == -1.0).sum()
    missing = col_data.isna().sum()
    
    label_stats.append({
        'Label': col,
        'Positive': positive,
        'Negative': negative, 
        'Uncertain': uncertain,
        'Missing': missing,
        'Positive%': positive/total*100,
        'Negative%': negative/total*100,
        'Uncertain%': uncertain/total*100,
        'Missing%': missing/total*100
    })

label_stats_df = pd.DataFrame(label_stats)
print("Label Distribution Statistics:")
label_stats_df

In [ ]:
# Visualize label distribution
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Positive cases
axes[0,0].barh(label_stats_df['Label'], label_stats_df['Positive%'])
axes[0,0].set_title('Positive Cases Distribution (%)')
axes[0,0].set_xlabel('Percentage')

# Uncertain cases
axes[0,1].barh(label_stats_df['Label'], label_stats_df['Uncertain%'], color='orange')
axes[0,1].set_title('Uncertain Cases Distribution (%)')
axes[0,1].set_xlabel('Percentage')

# Missing cases
axes[1,0].barh(label_stats_df['Label'], label_stats_df['Missing%'], color='red')
axes[1,0].set_title('Missing Cases Distribution (%)')
axes[1,0].set_xlabel('Percentage')

# Combined stacked bar chart
x = range(len(pathology_columns))
width = 0.8

axes[1,1].barh(x, label_stats_df['Positive%'], width, label='Positive', color='green', alpha=0.8)
axes[1,1].barh(x, label_stats_df['Negative%'], width, left=label_stats_df['Positive%'], 
              label='Negative', color='blue', alpha=0.8)
axes[1,1].barh(x, label_stats_df['Uncertain%'], width, 
              left=label_stats_df['Positive%']+label_stats_df['Negative%'], 
              label='Uncertain', color='orange', alpha=0.8)
axes[1,1].barh(x, label_stats_df['Missing%'], width,
              left=label_stats_df['Positive%']+label_stats_df['Negative%']+label_stats_df['Uncertain%'],
              label='Missing', color='red', alpha=0.8)

axes[1,1].set_yticks(x)
axes[1,1].set_yticklabels(label_stats_df['Label'])
axes[1,1].set_xlabel('Percentage')
axes[1,1].set_title('Complete Label Distribution')
axes[1,1].legend()

plt.tight_layout()
plt.show()

## 4. Image Analysis

In [ ]:
# Sample some images for analysis
sample_paths = train_df['Path'].sample(10, random_state=42)

# Analyze image properties
image_info = []

for path in sample_paths:
    full_path = DATA_ROOT / path
    if full_path.exists():
        try:
            img = Image.open(full_path)
            image_info.append({
                'Path': path,
                'Width': img.width,
                'Height': img.height,
                'Mode': img.mode,
                'Format': img.format,
                'Size_MB': full_path.stat().st_size / (1024*1024)
            })
        except Exception as e:
            print(f"Error loading {path}: {e}")
    else:
        print(f"Path does not exist: {full_path}")

image_info_df = pd.DataFrame(image_info)
print("Sample Image Properties:")
image_info_df

In [ ]:
# Image statistics
if not image_info_df.empty:
    print("Image Statistics:")
    print(f"Width range: {image_info_df['Width'].min()} - {image_info_df['Width'].max()}")
    print(f"Height range: {image_info_df['Height'].min()} - {image_info_df['Height'].max()}")
    print(f"Average size: {image_info_df['Size_MB'].mean():.2f} MB")
    print(f"Modes: {image_info_df['Mode'].unique()}")
    print(f"Formats: {image_info_df['Format'].unique()}")
    
    # Plot image size distribution
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].hist(image_info_df['Width'], bins=10, alpha=0.7)
    axes[0].set_title('Image Width Distribution')
    axes[0].set_xlabel('Width (pixels)')
    
    axes[1].hist(image_info_df['Height'], bins=10, alpha=0.7)
    axes[1].set_title('Image Height Distribution')
    axes[1].set_xlabel('Height (pixels)')
    
    axes[2].hist(image_info_df['Size_MB'], bins=10, alpha=0.7)
    axes[2].set_title('File Size Distribution')
    axes[2].set_xlabel('Size (MB)')
    
    plt.tight_layout()
    plt.show()

## 5. Sample Image Visualization

In [ ]:
# Visualize sample images with their labels
def plot_sample_images(df, n_samples=6):
    """Plot sample images with their pathology labels"""
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()
    
    sample_indices = np.random.choice(len(df), n_samples, replace=False)
    
    for i, idx in enumerate(sample_indices):
        row = df.iloc[idx]
        img_path = DATA_ROOT / row['Path']
        
        if img_path.exists():
            try:
                img = Image.open(img_path).convert('L')  # Convert to grayscale
                axes[i].imshow(img, cmap='gray')
                axes[i].axis('off')
                
                # Create title with pathology labels
                positive_labels = []
                for label in pathology_columns:
                    if row[label] == 1.0:
                        positive_labels.append(label.replace(' ', '\n'))
                
                title = f"Age: {row['Age']}, Sex: {row['Sex']}\n"
                if positive_labels:
                    title += "Findings: " + ", ".join(positive_labels[:3])  # Show first 3
                    if len(positive_labels) > 3:
                        title += f" (+{len(positive_labels)-3} more)"
                else:
                    title += "No positive findings"
                
                axes[i].set_title(title, fontsize=8)
            except Exception as e:
                axes[i].text(0.5, 0.5, f'Error loading\n{str(e)}', 
                           ha='center', va='center', transform=axes[i].transAxes)
                axes[i].axis('off')
        else:
            axes[i].text(0.5, 0.5, 'Image not found', 
                       ha='center', va='center', transform=axes[i].transAxes)
            axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Plot sample images
plot_sample_images(train_df)

## 6. Uncertainty Analysis

In [ ]:
# Analyze the uncertainty patterns
print("Uncertainty Analysis:")
print("="*50)

# Total uncertain cases per pathology
uncertain_counts = (pathology_data == -1.0).sum().sort_values(ascending=False)
print("\nPathologies with most uncertain labels:")
for label, count in uncertain_counts.head(10).items():
    percentage = count / len(pathology_data) * 100
    print(f"{label:25s}: {count:6d} ({percentage:5.1f}%)")

# Samples with multiple uncertain labels
uncertain_per_sample = (pathology_data == -1.0).sum(axis=1)
print(f"\nSamples with uncertain labels:")
print(f"No uncertain labels: {(uncertain_per_sample == 0).sum():6d} ({(uncertain_per_sample == 0).mean()*100:5.1f}%)")
print(f"1+ uncertain labels: {(uncertain_per_sample > 0).sum():6d} ({(uncertain_per_sample > 0).mean()*100:5.1f}%)")
print(f"5+ uncertain labels: {(uncertain_per_sample >= 5).sum():6d} ({(uncertain_per_sample >= 5).mean()*100:5.1f}%)")
print(f"Max uncertain per sample: {uncertain_per_sample.max()}")

# Plot uncertainty distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
uncertain_counts.plot(kind='barh')
plt.title('Uncertain Labels by Pathology')
plt.xlabel('Count')

plt.subplot(1, 2, 2)
plt.hist(uncertain_per_sample, bins=range(15), alpha=0.7)
plt.title('Distribution of Uncertain Labels per Sample')
plt.xlabel('Number of Uncertain Labels')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

## 7. Class Imbalance Analysis

In [ ]:
# Calculate class imbalance ratios
imbalance_stats = []

for col in pathology_columns:
    col_data = pathology_data[col]
    
    # Consider only non-missing, non-uncertain labels for imbalance calculation
    valid_data = col_data[(col_data == 0.0) | (col_data == 1.0)]
    
    if len(valid_data) > 0:
        positive = (valid_data == 1.0).sum()
        negative = (valid_data == 0.0).sum()
        
        pos_ratio = positive / len(valid_data)
        imbalance_ratio = negative / positive if positive > 0 else float('inf')
        
        imbalance_stats.append({
            'Label': col,
            'Positive': positive,
            'Negative': negative,
            'Total_Valid': len(valid_data),
            'Positive_Ratio': pos_ratio,
            'Imbalance_Ratio': imbalance_ratio
        })

imbalance_df = pd.DataFrame(imbalance_stats)
imbalance_df = imbalance_df.sort_values('Imbalance_Ratio', ascending=False)

print("Class Imbalance Analysis:")
print("(Imbalance Ratio = Negative / Positive)")
print("\nMost imbalanced classes:")
imbalance_df.head(10)

In [ ]:
# Visualize class imbalance
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Positive ratio
axes[0].barh(imbalance_df['Label'], imbalance_df['Positive_Ratio'])
axes[0].set_title('Positive Class Ratio by Pathology')
axes[0].set_xlabel('Positive Ratio')
axes[0].axvline(x=0.5, color='red', linestyle='--', alpha=0.7, label='Balanced (0.5)')
axes[0].legend()

# Imbalance ratio (log scale)
finite_ratios = imbalance_df[imbalance_df['Imbalance_Ratio'] != float('inf')]
axes[1].barh(finite_ratios['Label'], np.log10(finite_ratios['Imbalance_Ratio']))
axes[1].set_title('Class Imbalance Ratio (log10 scale)')
axes[1].set_xlabel('log10(Negative/Positive)')
axes[1].axvline(x=0, color='red', linestyle='--', alpha=0.7, label='Balanced (log10(1)=0)')
axes[1].legend()

plt.tight_layout()
plt.show()

# Summary statistics
print(f"\nImbalance Summary:")
print(f"Most balanced class: {imbalance_df.loc[imbalance_df['Imbalance_Ratio'].idxmin(), 'Label']} (ratio: {imbalance_df['Imbalance_Ratio'].min():.2f})")
print(f"Most imbalanced class: {imbalance_df.loc[imbalance_df['Imbalance_Ratio'].idxmax(), 'Label']} (ratio: {imbalance_df['Imbalance_Ratio'].max():.2f})")
print(f"Average positive ratio: {imbalance_df['Positive_Ratio'].mean():.3f}")
print(f"Median positive ratio: {imbalance_df['Positive_Ratio'].median():.3f}")

## 8. Co-occurrence Analysis

In [ ]:
# Analyze pathology co-occurrence patterns
# Convert uncertain labels to 0 for this analysis
binary_labels = pathology_data.copy()
binary_labels[binary_labels == -1.0] = 0.0  # Treat uncertain as negative
binary_labels = binary_labels.fillna(0.0)    # Treat missing as negative

# Calculate correlation matrix
correlation_matrix = binary_labels.corr()

# Plot correlation heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))  # Mask upper triangle
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.2f', cbar_kws={"shrink": .8})
plt.title('Pathology Co-occurrence Correlation Matrix')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Find strongest correlations
correlation_pairs = []
for i in range(len(pathology_columns)):
    for j in range(i+1, len(pathology_columns)):
        corr = correlation_matrix.iloc[i, j]
        correlation_pairs.append({
            'Pathology_1': pathology_columns[i],
            'Pathology_2': pathology_columns[j],
            'Correlation': corr
        })

correlation_pairs_df = pd.DataFrame(correlation_pairs)
correlation_pairs_df = correlation_pairs_df.sort_values('Correlation', key=abs, ascending=False)

print("\nStrongest pathology correlations:")
print(correlation_pairs_df.head(10))

## 9. Summary and Recommendations

In [ ]:
# Summary statistics and recommendations
print("DiagXNet-Lite: Dataset Analysis Summary")
print("="*50)

print(f"\n📊 Dataset Overview:")
print(f"   • Total samples: {len(train_df):,}")
print(f"   • Pathology labels: {len(pathology_columns)}")
print(f"   • Patient age range: {train_df['Age'].min():.0f}-{train_df['Age'].max():.0f} years")
print(f"   • View types: {train_df['Frontal/Lateral'].value_counts().to_dict()}")

print(f"\n🏷️ Label Characteristics:")
total_uncertain = (pathology_data == -1.0).sum().sum()
total_missing = pathology_data.isna().sum().sum()
total_positive = (pathology_data == 1.0).sum().sum()
print(f"   • Total uncertain labels: {total_uncertain:,} ({total_uncertain/(len(train_df)*len(pathology_columns))*100:.1f}%)")
print(f"   • Total missing labels: {total_missing:,} ({total_missing/(len(train_df)*len(pathology_columns))*100:.1f}%)")
print(f"   • Total positive labels: {total_positive:,} ({total_positive/(len(train_df)*len(pathology_columns))*100:.1f}%)")
print(f"   • Most common pathology: {label_stats_df.loc[label_stats_df['Positive'].idxmax(), 'Label']}")
print(f"   • Least common pathology: {label_stats_df.loc[label_stats_df['Positive'].idxmin(), 'Label']}")

print(f"\n⚖️ Class Imbalance:")
balanced_threshold = 0.1  # Consider classes with 10-90% positive ratio as relatively balanced
balanced_classes = imbalance_df[(imbalance_df['Positive_Ratio'] >= balanced_threshold) & 
                               (imbalance_df['Positive_Ratio'] <= 1-balanced_threshold)]
print(f"   • Relatively balanced classes: {len(balanced_classes)}/{len(imbalance_df)}")
print(f"   • Most imbalanced: {imbalance_df.iloc[0]['Label']} (ratio: {imbalance_df.iloc[0]['Imbalance_Ratio']:.1f}:1)")
print(f"   • Average positive ratio: {imbalance_df['Positive_Ratio'].mean():.3f}")

print(f"\n🔗 Co-occurrence Patterns:")
strong_correlations = correlation_pairs_df[abs(correlation_pairs_df['Correlation']) > 0.3]
print(f"   • Strong correlations (>0.3): {len(strong_correlations)}")
if len(strong_correlations) > 0:
    top_corr = correlation_pairs_df.iloc[0]
    print(f"   • Strongest correlation: {top_corr['Pathology_1']} ↔ {top_corr['Pathology_2']} ({top_corr['Correlation']:.3f})")

print(f"\n💡 Recommendations for Model Development:")
print(f"   1. Handle uncertain labels carefully - consider U-Ignore or U-Zeros strategies")
print(f"   2. Address class imbalance with weighted loss functions or sampling strategies")
print(f"   3. Consider multi-label classification metrics (AUC-ROC, AUC-PR per class)")
print(f"   4. Leverage pathology correlations for multi-task learning")
print(f"   5. Implement data augmentation to increase dataset diversity")
print(f"   6. Use stratified sampling to maintain label distribution in train/val splits")

print(f"\n🎯 Next Steps:")
print(f"   1. Implement data preprocessing pipeline")
print(f"   2. Create baseline model evaluation")
print(f"   3. Develop custom architectures for multi-label classification")
print(f"   4. Implement uncertainty-aware training strategies")
print(f"   5. Add interpretability analysis (GradCAM, attention maps)")